In [ ]:
import xarray as xr
import gcsfs
import pickle

%matplotlib inline

In [ ]:
das = {}
das['maroni'] = xr.open_zarr('../data/maroni_mask')
das['oyapock'] = xr.open_zarr('../data/oyapock_mask')

In [ ]:
def get_ws_p(pix_deg, da_masks, da_p, tolerance=None):
    if tolerance is None:
        s = str(pix_deg)
        tolerance = 10 ** (-(len(s) - s.find('.')))
    with open(f'../data/pix_deg_{pix_deg}.pkl', 'rb') as f:
        da_area = pickle.load(f)
    da_masks = da_area.reindex_like(da_masks, method='nearest', tolerance=tolerance) * da_masks
    da_masks = da_masks / da_masks.sum(['lat', 'lon'])
    p = (da_p.reindex_like(da_masks, method='nearest', tolerance=tolerance).clip(0) * da_masks).sum(['lat', 'lon'])
    return p

In [ ]:
ds_p = xr.open_zarr(gcsfs.GCSMap('pangeo-data/gpm_imerg_early'))
ds_p

In [ ]:
da = xr.concat(list(das.values()), 'label')
da_masks = da.assign_coords(label=list(das))

In [ ]:
from_time, to_time = '2014-03-12', '2019-04-01'
ds_p = ds_p.sel(time=slice(from_time, to_time))
da_p = ds_p['precipitationCal']
p = get_ws_p(0.1, da_masks, da_p).chunk({'time': -1, 'label': 1})
p

In [ ]:
p = p['mask'].compute()